In [11]:
from astropy.io import fits
from radio_beam import Beam
from astropy.wcs import WCS
from radio_beam import Beams
from astropy.convolution import convolve
from spectral_cube import SpectralCube
import matplotlib.pyplot as plt
from matplotlib.patches import Ellipse
from astropy import units as u
import radio_beam
W51b6 = '/orange/adamginsburg/w51/TaehwaYoo/2015.1.01596.S_W51_B6_LB/continuum_images/'
W51cont='/orange/adamginsburg/w51/TaehwaYoo/b6contfits/'
w51conv = '/orange/adamginsburg/w51/TaehwaYoo/convolved_new/'
w51e2_b6_briggs=W51cont+'W51e2_cont_bigbriggs.image.fits'
w51e2_b6_robust0=W51cont+'W51e2_cont_big_robust0.image.fits'
w51e2_b6_uniform=W51cont+'W51e2_cont_biguniform.image.fits'
w51e2_b6_superuniform=W51cont+'W51e2_cont_bigsuperuniform.image.fits'

w51n_b6_briggs = W51cont+'W51n_cont_bigbriggs.image.fits'
w51n_b6_robust0 = W51cont+'w51n_cont_big_robust0.image.fits'
w51n_b6_uniform = W51cont+'W51n_cont_biguniform.image.fits'
w51n_b6_superuniform = W51cont+'W51n_cont_bigsuperuniform.image.fits'
w51n_b6_natural = W51cont+'W51n_cont_bignatural.image.fits'

W51b3 = '/orange/adamginsburg/w51/TaehwaYoo/2017.1.00293.S_W51_B3_LB/may2021_successful_imaging/'
w51n_b3_tt0 = W51b3+'w51n.spw0thru19.14500.robust0.thr0.075mJy.mfs.I.startmod.selfcal7.image.tt0.pbcor.fits'
w51e2_b3_tt0 = W51b3+'w51e2.spw0thru19.14500.robust0.thr0.075mJy.mfs.I.startmod.selfcal7.image.tt0.pbcor.fits'
w51_alma_imf = '/orange/adamginsburg/w51/TaehwaYoo/w51_alma_imf/'
w51e_b3_alma_imf = '/orange/adamginsburg/w51/TaehwaYoo/w51_alma_imf/W51-E_B3_uid___A001_X1296_X10b_continuum_merged_12M_robust0_selfcal7_finaliter.image.tt0.pbcor.fits'
w51e_b6_alma_imf = '/orange/adamginsburg/w51/TaehwaYoo/w51_alma_imf/W51-E_B6_uid___A001_X1296_X213_continuum_merged_12M_robust0_selfcal7_finaliter.image.tt0.pbcor.fits'
w51n_b3_alma_imf = '/orange/adamginsburg/w51/TaehwaYoo/w51_alma_imf/W51-IRS2_B3_uid___A001_X1296_X18f_continuum_merged_12M_robust0_selfcal4_finaliter.image.tt0.pbcor.fits'
w51n_b6_alma_imf = '/orange/adamginsburg/w51/TaehwaYoo/w51_alma_imf/W51-IRS2_B6_uid___A001_X1296_X187_continuum_merged_12M_robust0_selfcal9_finaliter.image.tt0.pbcor.fits'









In [18]:
def convolve_adam(b3file, b6file,outdir,label):
    image_b3 = SpectralCube.read(b3file, use_dask=False, format='fits',allow_huge_operations=True, how='slice').minimal_subcube()
    image_b6 = SpectralCube.read(b6file, use_dask=False, format='fits',allow_huge_operations=True, how='slice').minimal_subcube()
    image_b3.allow_huge_operations=True
    image_b6.allow_huge_operations=True

    # Jy/beam -> Jy/sr
    image_b3 = image_b3 * u.beam / image_b3.beam.sr
    image_b6 = image_b6 * u.beam / image_b6.beam.sr
    
    hdrNB6 = fits.getheader(b6file)  
    hdrNB3 = fits.getheader(b3file)  

    beams = radio_beam.Beams(major=u.Quantity([image_b3.beam.major, image_b6.beam.major]),
                                 minor=u.Quantity([image_b3.beam.minor, image_b6.beam.minor]),
                                 pa=u.Quantity([image_b3.beam.pa, image_b6.beam.pa]))
    commonbeam = radio_beam.commonbeam.commonbeam(beams)
    
    if image_b3.beam.sr < image_b6.beam.sr:
        header = image_b6[0].header
    else:
        header = image_b3[0].header
    image_b3_repr = image_b3[0].convolve_to(commonbeam, allow_huge=True).reproject(header)
    fits.writeto(outdir+'/%s_B3_conv_adam.fits'%label, image_b3_repr.value, header, overwrite = True)

    image_b6_repr = image_b6[0].convolve_to(commonbeam, allow_huge=True).reproject(header)
    
    fits.writeto(outdir+'/%s_B6_conv_adam.fits'%label, image_b6_repr.value, header, overwrite = True)

    
    


In [19]:
convolve_adam(w51n_b3_tt0,w51n_b6_briggs,w51conv,'w51n')
convolve_adam(w51e2_b3_tt0,w51e2_b6_briggs,w51conv,'w51e')


/home/t.yoo/anaconda3/lib/python3.9/site-packages/spectral_cube/lower_dimensional_structures.py:462: UserWarning: The given beam is identical to the current beam. Skipping convolution.
  warnings.warn("The given beam is identical to the current beam. "
/home/t.yoo/anaconda3/lib/python3.9/site-packages/spectral_cube/lower_dimensional_structures.py:462: UserWarning: The given beam is identical to the current beam. Skipping convolution.
  warnings.warn("The given beam is identical to the current beam. "


In [10]:
convolve_adam(w51e_b3_alma_imf, w51e_b6_alma_imf, w51_alma_imf, 'w51e')
convolve_adam(w51n_b3_alma_imf, w51n_b6_alma_imf, w51_alma_imf, 'w51n')

/home/t.yoo/anaconda3/lib/python3.9/site-packages/spectral_cube/lower_dimensional_structures.py:462: UserWarning: The given beam is identical to the current beam. Skipping convolution.
  warnings.warn("The given beam is identical to the current beam. "


In [ ]:
def convolve_adam(b3file, b6file,outdir,label):
    image_b3 = SpectralCube.read(b3file, use_dask=False, format='fits').minimal_subcube()
    image_b6 = SpectralCube.read(b6file, use_dask=False, format='fits').minimal_subcube()
    image_b3 = image_b3 * u.beam / image_b3.beam.sr
    image_b6 = image_b6 * u.beam / image_b6.beam.sr
    
    hdrNB6 = fits.getheader(b6file)  
    hdrNB3 = fits.getheader(b3file)  

    beams = radio_beam.Beams(major=u.Quantity([image_b3.beam.major, image_b6.beam.major]),
                                 minor=u.Quantity([image_b3.beam.minor, image_b6.beam.minor]),
                                 pa=u.Quantity([image_b3.beam.pa, image_b6.beam.pa]))
    commonbeam = radio_beam.commonbeam.commonbeam(beams)
    
    if image_b3.beam.sr < image_b6.beam.sr:
        header = image_b6[0].header
    else:
        header = image_b3[0].header
    image_b3_repr = image_b3[0].convolve_to(commonbeam, allow_huge=True).reproject(header)
    #fits.writeto(outdir+'/%s_B3_conv_adam.fits'%label, image_b3_repr.value, header, overwrite = True)

    image_b6_repr = image_b6[0].convolve_to(commonbeam, allow_huge=True).reproject(header)
    
    
    
    #fits.writeto(outdir+'/%s_B6_conv_adam.fits'%label, image_b6_repr.value, header, overwrite = True)

    
    
